In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyts.image import GramianAngularField
from PIL import Image
import os
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, classification_report, accuracy_score

#### The Human activity recognition dataset is a multiclass classification dataset with sensor data. 3 sensors give 9 timeseries signals as output. https://archive.ics.uci.edu/ml/datasets/human+activity+recognition+using+smartphones

## Data Preparation

In [8]:
files = os.listdir('./HARDataset/train/Inertial Signals/')
files.sort()
files

['body_acc_x_train.txt',
 'body_acc_y_train.txt',
 'body_acc_z_train.txt',
 'body_gyro_x_train.txt',
 'body_gyro_y_train.txt',
 'body_gyro_z_train.txt',
 'total_acc_x_train.txt',
 'total_acc_y_train.txt',
 'total_acc_z_train.txt']

In [13]:
y = pd.read_csv('./HARDataset/train/y_train.txt', header=None, delim_whitespace=True).values
y.shape

(7352, 1)

In [11]:
acc_x = pd.read_csv('./HARDataset/train/Inertial Signals/body_acc_x_train.txt', header=None, delim_whitespace=True).values
acc_y = pd.read_csv('./HARDataset/train/Inertial Signals/body_acc_y_train.txt', header=None, delim_whitespace=True).values
acc_z = pd.read_csv('./HARDataset/train/Inertial Signals/body_acc_z_train.txt', header=None, delim_whitespace=True).values

acc_x.shape,acc_y.shape,acc_z.shape

((7352, 128), (7352, 128), (7352, 128))

In [24]:
gyro_x = pd.read_csv('./HARDataset/train/Inertial Signals/body_gyro_x_train.txt', header=None, delim_whitespace=True).values
gyro_y = pd.read_csv('./HARDataset/train/Inertial Signals/body_gyro_y_train.txt', header=None, delim_whitespace=True).values
gyro_z = pd.read_csv('./HARDataset/train/Inertial Signals/body_gyro_z_train.txt', header=None, delim_whitespace=True).values

gyro_x.shape,gyro_y.shape,gyro_z.shape

((7352, 128), (7352, 128), (7352, 128))

In [25]:
total_x = pd.read_csv('./HARDataset/train/Inertial Signals/total_acc_x_train.txt', header=None, delim_whitespace=True).values
total_y = pd.read_csv('./HARDataset/train/Inertial Signals/total_acc_y_train.txt', header=None, delim_whitespace=True).values
total_z = pd.read_csv('./HARDataset/train/Inertial Signals/total_acc_z_train.txt', header=None, delim_whitespace=True).values

total_x.shape,total_y.shape,total_z.shape

((7352, 128), (7352, 128), (7352, 128))

### Generate Images 
We use gramarian angular field based images

In [14]:
!mkdir 'images'

In [38]:
gaf = GramianAngularField()
filenames = []
for i in tqdm(range(len(y))):
    img_acc_x = gaf.fit_transform(acc_x[i].reshape((-1,128)))[0]
    img_acc_y = gaf.fit_transform(acc_y[i].reshape((-1,128)))[0]
    img_acc_z = gaf.fit_transform(acc_z[i].reshape((-1,128)))[0]
    
    img_gyro_x = gaf.fit_transform(gyro_x[i].reshape((-1,128)))[0]
    img_gyro_y = gaf.fit_transform(gyro_y[i].reshape((-1,128)))[0]
    img_gyro_z = gaf.fit_transform(gyro_z[i].reshape((-1,128)))[0]
    
    img_total_x = gaf.fit_transform(total_x[i].reshape((-1,128)))[0]
    img_total_y = gaf.fit_transform(total_y[i].reshape((-1,128)))[0]
    img_total_z = gaf.fit_transform(total_z[i].reshape((-1,128)))[0]
    
    img_acc = np.concatenate((img_acc_x,img_acc_y,img_acc_z), axis=1)
    img_gyro = np.concatenate((img_gyro_x,img_gyro_y,img_gyro_z), axis=1)
    img_total = np.concatenate((img_total_x,img_total_y,img_total_z), axis=1)
    img = np.concatenate((img_acc, img_gyro, img_total), axis=0)
    img = ((img + 1) / (2) ) * (255 - 0) + 0
    
    image = Image.fromarray(img)
    image = image.convert('RGB')
    image = image.resize((224,224))
    image.save('./images/'+str(i)+'.png')
    filenames.append('./images/'+str(i)+'.png')
print(len(filenames))

100%|██████████| 7352/7352 [03:43<00:00, 32.86it/s]

7352


In [9]:
dict_activity = {1:'WALKING',
2:'WALKING_UPSTAIRS',
3:'WALKING_DOWNSTAIRS',
4:'SITTING',
5:'STANDING',
6:'LAYING'}

def get_label(val):
    return dict_activity.get(val)

In [48]:
df_subject = pd.read_csv('./HARDataset/train/subject_train.txt', header=None, delim_whitespace=True)
df_subject.columns = ['subject_id']
df_dr = pd.read_csv('./HARDataset/train/y_train.txt', header=None, delim_whitespace=True)
df_dr.columns = ['target']
df_dr['image'] = filenames
df_dr['subject_id'] = df_subject['subject_id']
df_dr['target_str'] =  df_dr.target.apply(get_label)
df_dr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7352 entries, 0 to 7351
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   target      7352 non-null   int64 
 1   image       7352 non-null   object
 2   subject_id  7352 non-null   int64 
 3   target_str  7352 non-null   object
dtypes: int64(2), object(2)
memory usage: 229.9+ KB


In [49]:
df_dr.head()

,target,image,subject_id,target_str
0,5,./images/0.png,1,STANDING
1,5,./images/1.png,1,STANDING
2,5,./images/2.png,1,STANDING
3,5,./images/3.png,1,STANDING
4,5,./images/4.png,1,STANDING


In [51]:
df_dr[['image', 'subject_id', 'target_str']].to_csv('./df_har.csv',index=False)

## Baseline Model
This model only uses aggregated, centralized features on the time series

In [36]:
y = pd.read_csv('./HARDataset/train/y_train.txt', header=None, delim_whitespace=True)
y.columns = ['target']
y['target_string'] = y.target.apply(get_label)
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7352 entries, 0 to 7351
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   target         7352 non-null   int64 
 1   target_string  7352 non-null   object
dtypes: int64(1), object(1)
memory usage: 115.0+ KB


In [56]:
df_subject = pd.read_csv('./HARDataset/train/subject_train.txt', header=None, delim_whitespace=True)
df_subject.columns = ['subject_id']

In [57]:
column_names = pd.read_csv('./HARDataset/features.txt', header=None, delim_whitespace=True)
column_names.columns = ['id','feature']
column_names.head()

,id,feature
0,1,tBodyAcc-mean()-X
1,2,tBodyAcc-mean()-Y
2,3,tBodyAcc-mean()-Z
3,4,tBodyAcc-std()-X
4,5,tBodyAcc-std()-Y


In [58]:
X = pd.read_csv('./HARDataset/train/X_train.txt', header=None, delim_whitespace=True)
# X.columns = column_names.feature.tolist()
X.columns = ['feat_'+str(i+1) for i in range(X.shape[1])]
X['target_str'] = y['target_string']
X['subject_id'] = df_subject['subject_id']
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7352 entries, 0 to 7351
Columns: 563 entries, feat_1 to subject_id
dtypes: float64(561), int64(1), object(1)
memory usage: 31.6+ MB


In [59]:
X.to_csv('./uci_har_tabular.csv', index=False)

In [65]:
X_test = pd.read_csv('./HARDataset/test/X_test.txt', header=None, delim_whitespace=True)
y_test = pd.read_csv('./HARDataset/test/y_test.txt', header=None, delim_whitespace=True)
y_test.columns = ['target']
y_test['target_string'] = y_test.target.apply(get_label)
df_subject_test = pd.read_csv('./HARDataset/test/subject_test.txt', header=None, delim_whitespace=True)
df_subject_test.columns = ['subject_id']

In [66]:
# X.columns = column_names.feature.tolist()
X_test.columns = ['feat_'+str(i+1) for i in range(X_test.shape[1])]
X_test['target_str'] = y_test['target_string']
X_test['subject_id'] = df_subject_test['subject_id']
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2947 entries, 0 to 2946
Columns: 563 entries, feat_1 to subject_id
dtypes: float64(561), int64(1), object(1)
memory usage: 12.7+ MB


In [67]:
X_test.to_csv('./uci_har_tabular_test.csv', index=False)

In [90]:
preds = pd.read_csv('./preds_blender_tabular.csv')
df_performance = preds.copy(deep=True)
df_performance.rename(columns={'Prediction LAYING':'LAYING',
                               'Prediction SITTING':'SITTING',
                               'Prediction STANDING':'STANDING',
                               'Prediction WALKING':'WALKING',
                               'Prediction WALKING_DOWNSTAIRS':'WALKING_DOWNSTAIRS', 
                               'Prediction WALKING_UPSTAIRS':'WALKING_UPSTAIRS'}, inplace=True)
df_performance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2947 entries, 0 to 2946
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   row_id              2947 non-null   int64  
 1   target_str          2947 non-null   object 
 2   LAYING              2947 non-null   float64
 3   SITTING             2947 non-null   float64
 4   STANDING            2947 non-null   float64
 5   WALKING             2947 non-null   float64
 6   WALKING_DOWNSTAIRS  2947 non-null   float64
 7   WALKING_UPSTAIRS    2947 non-null   float64
dtypes: float64(6), int64(1), object(1)
memory usage: 184.3+ KB


In [91]:
roc_auc_score(df_performance.target_str, df_performance[['LAYING', 'SITTING', 'STANDING', 'WALKING',
       'WALKING_DOWNSTAIRS', 'WALKING_UPSTAIRS']], multi_class="ovr")

0.9975473922004675

In [92]:
labs = ['LAYING', 'SITTING', 'STANDING', 'WALKING',
       'WALKING_DOWNSTAIRS', 'WALKING_UPSTAIRS']
preds = [labs[i] for i in df_performance[['LAYING', 'SITTING', 'STANDING', 'WALKING',
       'WALKING_DOWNSTAIRS', 'WALKING_UPSTAIRS']].values.argmax(axis=1)]
print(classification_report(df_performance.target_str, preds))

                    precision    recall  f1-score   support

            LAYING       1.00      1.00      1.00       537
           SITTING       0.96      0.89      0.92       491
          STANDING       0.90      0.96      0.93       532
           WALKING       0.93      0.98      0.96       496
WALKING_DOWNSTAIRS       0.97      0.93      0.95       420
  WALKING_UPSTAIRS       0.93      0.92      0.92       471

          accuracy                           0.95      2947
         macro avg       0.95      0.95      0.95      2947
      weighted avg       0.95      0.95      0.95      2947



## Only TS Images - Visual AI model
use only the images as features and build visual ai models

In [2]:
files_test = os.listdir('./HARDataset/test/Inertial Signals/')
files_test.sort()
files_test

['body_acc_x_test.txt',
 'body_acc_y_test.txt',
 'body_acc_z_test.txt',
 'body_gyro_x_test.txt',
 'body_gyro_y_test.txt',
 'body_gyro_z_test.txt',
 'total_acc_x_test.txt',
 'total_acc_y_test.txt',
 'total_acc_z_test.txt']

In [3]:
acc_x = pd.read_csv('./HARDataset/test/Inertial Signals/body_acc_x_test.txt', header=None, delim_whitespace=True).values
acc_y = pd.read_csv('./HARDataset/test/Inertial Signals/body_acc_y_test.txt', header=None, delim_whitespace=True).values
acc_z = pd.read_csv('./HARDataset/test/Inertial Signals/body_acc_z_test.txt', header=None, delim_whitespace=True).values

acc_x.shape,acc_y.shape,acc_z.shape

((2947, 128), (2947, 128), (2947, 128))

In [4]:
gyro_x = pd.read_csv('./HARDataset/test/Inertial Signals/body_gyro_x_test.txt', header=None, delim_whitespace=True).values
gyro_y = pd.read_csv('./HARDataset/test/Inertial Signals/body_gyro_y_test.txt', header=None, delim_whitespace=True).values
gyro_z = pd.read_csv('./HARDataset/test/Inertial Signals/body_gyro_z_test.txt', header=None, delim_whitespace=True).values

gyro_x.shape,gyro_y.shape,gyro_z.shape

((2947, 128), (2947, 128), (2947, 128))

In [5]:
total_x = pd.read_csv('./HARDataset/test/Inertial Signals/total_acc_x_test.txt', header=None, delim_whitespace=True).values
total_y = pd.read_csv('./HARDataset/test/Inertial Signals/total_acc_y_test.txt', header=None, delim_whitespace=True).values
total_z = pd.read_csv('./HARDataset/test/Inertial Signals/total_acc_z_test.txt', header=None, delim_whitespace=True).values

total_x.shape,total_y.shape,total_z.shape

((2947, 128), (2947, 128), (2947, 128))

In [6]:
!mkdir 'images_test'

In [8]:
y = pd.read_csv('./HARDataset/test/y_test.txt', header=None, delim_whitespace=True).values
y.shape

(2947, 1)

In [10]:
gaf = GramianAngularField()
filenames = []
for i in tqdm(range(len(y))):
    img_acc_x = gaf.fit_transform(acc_x[i].reshape((-1,128)))[0]
    img_acc_y = gaf.fit_transform(acc_y[i].reshape((-1,128)))[0]
    img_acc_z = gaf.fit_transform(acc_z[i].reshape((-1,128)))[0]
    
    img_gyro_x = gaf.fit_transform(gyro_x[i].reshape((-1,128)))[0]
    img_gyro_y = gaf.fit_transform(gyro_y[i].reshape((-1,128)))[0]
    img_gyro_z = gaf.fit_transform(gyro_z[i].reshape((-1,128)))[0]
    
    img_total_x = gaf.fit_transform(total_x[i].reshape((-1,128)))[0]
    img_total_y = gaf.fit_transform(total_y[i].reshape((-1,128)))[0]
    img_total_z = gaf.fit_transform(total_z[i].reshape((-1,128)))[0]
    
    img_acc = np.concatenate((img_acc_x,img_acc_y,img_acc_z), axis=1)
    img_gyro = np.concatenate((img_gyro_x,img_gyro_y,img_gyro_z), axis=1)
    img_total = np.concatenate((img_total_x,img_total_y,img_total_z), axis=1)
    img = np.concatenate((img_acc, img_gyro, img_total), axis=0)
    img = ((img + 1) / (2) ) * (255 - 0) + 0
    
    image = Image.fromarray(img)
    image = image.convert('RGB')
    image = image.resize((224,224))
    image.save('./images_test/'+str(i)+'.png')
    filenames.append('./images_test/'+str(i)+'.png')
print(len(filenames))

100%|██████████| 2947/2947 [01:49<00:00, 26.94it/s]

2947


In [80]:
df_subject = pd.read_csv('./HARDataset/test/subject_test.txt', header=None, delim_whitespace=True)
df_subject.columns = ['subject_id']
df_dr = pd.read_csv('./HARDataset/test/y_test.txt', header=None, delim_whitespace=True)
df_dr.columns = ['target']
df_dr['image'] = filenames
df_dr['subject_id'] = df_subject['subject_id']
df_dr['target_str'] =  df_dr.target.apply(get_label)
df_dr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2947 entries, 0 to 2946
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   target      2947 non-null   int64 
 1   image       2947 non-null   object
 2   subject_id  2947 non-null   int64 
 3   target_str  2947 non-null   object
dtypes: int64(2), object(2)
memory usage: 92.2+ KB


In [81]:
df_dr.head()

,target,image,subject_id,target_str
0,5,./images_test/0.png,2,STANDING
1,5,./images_test/1.png,2,STANDING
2,5,./images_test/2.png,2,STANDING
3,5,./images_test/3.png,2,STANDING
4,5,./images_test/4.png,2,STANDING


In [82]:
df_dr[['image', 'subject_id', 'target_str']].to_csv('./df_har_test.csv',index=False)

In [70]:
# preds = pd.read_csv('./preds_ensemble_80.csv')
preds = pd.read_csv('./preds_blender_100.csv')
preds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2947 entries, 0 to 2946
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   row_id                         2947 non-null   int64  
 1   target_str                     2947 non-null   object 
 2   Prediction LAYING              2947 non-null   float64
 3   Prediction SITTING             2947 non-null   float64
 4   Prediction STANDING            2947 non-null   float64
 5   Prediction WALKING             2947 non-null   float64
 6   Prediction WALKING_DOWNSTAIRS  2947 non-null   float64
 7   Prediction WALKING_UPSTAIRS    2947 non-null   float64
dtypes: float64(6), int64(1), object(1)
memory usage: 184.3+ KB


In [71]:
df_performance = preds.copy(deep=True) #  0.81
df_performance.rename(columns={'Prediction LAYING':'LAYING',
                               'Prediction SITTING':'SITTING',
                               'Prediction STANDING':'STANDING',
                               'Prediction WALKING':'WALKING',
                               'Prediction WALKING_DOWNSTAIRS':'WALKING_DOWNSTAIRS', 
                               'Prediction WALKING_UPSTAIRS':'WALKING_UPSTAIRS'}, inplace=True)
df_performance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2947 entries, 0 to 2946
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   row_id              2947 non-null   int64  
 1   target_str          2947 non-null   object 
 2   LAYING              2947 non-null   float64
 3   SITTING             2947 non-null   float64
 4   STANDING            2947 non-null   float64
 5   WALKING             2947 non-null   float64
 6   WALKING_DOWNSTAIRS  2947 non-null   float64
 7   WALKING_UPSTAIRS    2947 non-null   float64
dtypes: float64(6), int64(1), object(1)
memory usage: 184.3+ KB


In [72]:
df_performance.columns

Index(['row_id', 'target_str', 'LAYING', 'SITTING', 'STANDING', 'WALKING',
       'WALKING_DOWNSTAIRS', 'WALKING_UPSTAIRS'],
      dtype='object')

In [73]:
roc_auc_score(df_performance.target_str, df_performance[['LAYING', 'SITTING', 'STANDING', 'WALKING',
       'WALKING_DOWNSTAIRS', 'WALKING_UPSTAIRS']], multi_class="ovr")

0.9756452836622634

In [74]:
labs = ['LAYING', 'SITTING', 'STANDING', 'WALKING',
       'WALKING_DOWNSTAIRS', 'WALKING_UPSTAIRS']
preds = [labs[i] for i in df_performance[['LAYING', 'SITTING', 'STANDING', 'WALKING',
       'WALKING_DOWNSTAIRS', 'WALKING_UPSTAIRS']].values.argmax(axis=1)]
print(classification_report(df_performance.target_str, preds))

                    precision    recall  f1-score   support

            LAYING       0.83      0.80      0.82       537
           SITTING       0.69      0.60      0.64       491
          STANDING       0.71      0.82      0.76       532
           WALKING       0.93      0.93      0.93       496
WALKING_DOWNSTAIRS       0.92      0.95      0.94       420
  WALKING_UPSTAIRS       0.95      0.92      0.94       471

          accuracy                           0.83      2947
         macro avg       0.84      0.84      0.84      2947
      weighted avg       0.83      0.83      0.83      2947



## Time Series central features and Images
Use both time series aggregated features and images in DataRobot

In [85]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7352 entries, 0 to 7351
Columns: 563 entries, feat_1 to subject_id
dtypes: float64(561), int64(1), object(1)
memory usage: 31.6+ MB


In [86]:
filenames = []
for i in tqdm(range(len(X))):
    filenames.append('./images/'+str(i)+'.png')
print(len(filenames))

100%|██████████| 7352/7352 [00:00<00:00, 1088552.77it/s]

7352


In [87]:
filenames[0:10]

['./images/0.png',
 './images/1.png',
 './images/2.png',
 './images/3.png',
 './images/4.png',
 './images/5.png',
 './images/6.png',
 './images/7.png',
 './images/8.png',
 './images/9.png']

In [88]:
X['image'] = filenames

In [89]:
X.to_csv('./uci_har_tabular_images.csv', index=False)

In [93]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2947 entries, 0 to 2946
Columns: 563 entries, feat_1 to subject_id
dtypes: float64(561), int64(1), object(1)
memory usage: 12.7+ MB


In [94]:
filenames = []
for i in tqdm(range(len(X_test))):
    filenames.append('./images_test/'+str(i)+'.png')
print(len(filenames))

100%|██████████| 2947/2947 [00:00<00:00, 687426.39it/s]

2947


In [95]:
X_test['image'] = filenames

In [96]:
X_test.to_csv('./uci_har_tabular_test_images.csv', index=False)

In [4]:
# preds = pd.read_csv('./preds_tab_plus_img.csv')
preds = pd.read_csv('./preds_low_level_features.csv')
df_performance = preds.copy(deep=True) 
df_performance.rename(columns={'Prediction LAYING':'LAYING',
                               'Prediction SITTING':'SITTING',
                               'Prediction STANDING':'STANDING',
                               'Prediction WALKING':'WALKING',
                               'Prediction WALKING_DOWNSTAIRS':'WALKING_DOWNSTAIRS', 
                               'Prediction WALKING_UPSTAIRS':'WALKING_UPSTAIRS'}, inplace=True)
df_performance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2947 entries, 0 to 2946
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   row_id              2947 non-null   int64  
 1   target_str          2947 non-null   object 
 2   LAYING              2947 non-null   float64
 3   SITTING             2947 non-null   float64
 4   STANDING            2947 non-null   float64
 5   WALKING             2947 non-null   float64
 6   WALKING_DOWNSTAIRS  2947 non-null   float64
 7   WALKING_UPSTAIRS    2947 non-null   float64
dtypes: float64(6), int64(1), object(1)
memory usage: 184.3+ KB


In [5]:
roc_auc_score(df_performance.target_str, df_performance[['LAYING', 'SITTING', 'STANDING', 'WALKING',
       'WALKING_DOWNSTAIRS', 'WALKING_UPSTAIRS']], multi_class="ovr")

0.9991187488093504

In [6]:
labs = ['LAYING', 'SITTING', 'STANDING', 'WALKING',
       'WALKING_DOWNSTAIRS', 'WALKING_UPSTAIRS']
preds = [labs[i] for i in df_performance[['LAYING', 'SITTING', 'STANDING', 'WALKING',
       'WALKING_DOWNSTAIRS', 'WALKING_UPSTAIRS']].values.argmax(axis=1)]
print(classification_report(df_performance.target_str, preds))

                    precision    recall  f1-score   support

            LAYING       1.00      1.00      1.00       537
           SITTING       0.96      0.90      0.93       491
          STANDING       0.91      0.97      0.94       532
           WALKING       0.96      0.99      0.98       496
WALKING_DOWNSTAIRS       0.99      0.97      0.98       420
  WALKING_UPSTAIRS       0.98      0.96      0.97       471

          accuracy                           0.97      2947
         macro avg       0.97      0.97      0.97      2947
      weighted avg       0.97      0.97      0.97      2947



In [7]:
'''                 precision    recall  f1-score   support

            LAYING       1.00      1.00      1.00       537
           SITTING       0.96      0.89      0.92       491
          STANDING       0.90      0.96      0.93       532
           WALKING       0.93      0.98      0.96       496
WALKING_DOWNSTAIRS       0.97      0.93      0.95       420
  WALKING_UPSTAIRS       0.93      0.92      0.92       471

          accuracy                           0.95      2947
         macro avg       0.95      0.95      0.95      2947
      weighted avg       0.95      0.95      0.95      2947 '''

'                 precision    recall  f1-score   support\n\n            LAYING       1.00      1.00      1.00       537\n           SITTING       0.96      0.89      0.92       491\n          STANDING       0.90      0.96      0.93       532\n           WALKING       0.93      0.98      0.96       496\nWALKING_DOWNSTAIRS       0.97      0.93      0.95       420\n  WALKING_UPSTAIRS       0.93      0.92      0.92       471\n\n          accuracy                           0.95      2947\n         macro avg       0.95      0.95      0.95      2947\n      weighted avg       0.95      0.95      0.95      2947 '

In [8]:
(0.97-0.95)*100/0.95

2.105263157894739